In [5]:
import os
import pandas as pd
import numpy as np
#%matplotlib inline
#import matplotlib.pyplot as plt
import re
import urllib.request
#from tensorflow.keras.preprocessing.sequence import pad_sequences

# Read all the needed datasets
dataset들 읽고 합치기

In [6]:
import glob

input_file = r'.\data' # csv파일들이 있는 디렉토리 위치
output_file = r'.\data\Harry Potter.csv' # 병합하고 저장하려는 파일명

allFile_list = glob.glob(os.path.join(input_file, 'Harry Potter *')) # glob함수로 sales_로 시작하는 파일들을 모은다
allData = [] # 읽어 들인 csv파일 내용을 저장할 빈 리스트를 하나 만든다
for file in allFile_list:
    df = pd.read_csv(file,sep=';', encoding='iso-8859-1') # for구문으로 csv파일들을 읽어 들인다
    df.columns = ['Character','Sentence'] # index를 Character, Sentence로 맞춘다.
    allData.append(df) # 빈 리스트에 읽어 들인 내용을 추가한다
dataCombine = pd.concat(allData, axis=0, ignore_index=True) # 리스트의 내용을 수직으로 병합. ignore_index=True는 인데스 값이 기존 순서를 무시하고 순서대로 정렬되도록 한다.
dataCombine.to_csv(output_file, index=False) # to_csv함수로 저장한다. 인데스를 빼려면 False로 설정

harry=pd.read_csv('./data/Harry Potter.csv')
harry

,Character,Sentence
0,Dumbledore,"I should've known that you would be here, Prof..."
1,McGonagall,"Good evening, Professor Dumbledore."
2,McGonagall,"Are the rumors true, Albus?"
3,Dumbledore,"I'm afraid so, professor."
4,Dumbledore,The good and the bad.
...,...,...
4920,HERMIONE,"How fast is it, Harry?"
4921,HARRY,Lumos.
4922,HARRY,I solemnly swear that I am up to no good.
4923,HARRY,Mischief managed.


# 등장인물 별 대사 모으기

In [7]:
# 등장인물의 이름이 unifying하지 않아서 전부 대문자로 바꿔주기
harry['Character'] = harry['Character'].str.upper()
# Character 별로 대사 개수 순으로 나열해서 가장 많은 5명 뽑기: groupby(묶어서), count(개수 세고), sort(나열하고), [-5:](뒤에서부터 5개)
selected = harry.groupby('Character').count().sort_values(by='Sentence')[-5:]   # Groupby의 label이었던 Character가 index가 된다.
topfive = [character for character in selected.index]   # 대사가 가장 많은 5명의 리스트 (역순)

for character in topfive:
    condition = harry['Character'] == character # condition: harry['Character']가 topfive의 원소인 character와 같을 bool 조건
    ownsentence = harry[condition]  # ownsentence: condition에 맞는 row만 filtering한 dataframe
    output_file = '.\data\\test\\HP_'+character+'.csv' # raw string이 아니라 '\'를 쓰려면 \\라고 해야 함
    ownsentence.to_csv(output_file, index=False)    # data\test\HP_(등장인물이름).csv 파일로 저장

# Stemming
morph analysis 기법.

In [8]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords


import glob 

stemmer=PorterStemmer()  

bag_of_words = {} #{등장인물 : 해당인물의 stemData, ~}
stemData = []
input_file = r'.\data\\test\\'

allHPfile_list = glob.glob(os.path.join(input_file, 'HP_*'))
print(allHPfile_list)

for file in allHPfile_list:
    df = pd.read_csv(file, sep=',', encoding='iso-8859-1')
    #print(df)
    c_df = df['Sentence'].str.replace('[^a-zA-Z ]' ,'')
    tokenData = nltk.word_tokenize(str(c_df))

    for w in tokenData:
        tempData=[]
        tempData=stemmer.stem(w)
        stemData.append(tempData)
    bag_of_words[df['Character'][0]] = stemData
    stemData=[]
print(bag_of_words)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
['.\\data\\\\test\\HP_DUMBLEDORE.csv', '.\\data\\\\test\\HP_HAGRID.csv', '.\\data\\\\test\\HP_HARRY.csv', '.\\data\\\\test\\HP_HERMIONE.csv', '.\\data\\\\test\\HP_RON.csv']


NameError: name 'stemmer' is not defined